In [266]:
import pandas as pd
import requests
import json

columns=['GEO_ID','FSRR2010']

responses = pd.DataFrame(columns=columns)

for i in range(1,57):
    if i not in [3,7,14,43,52]:
        if i < 10:
            url = "https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&for=tract:*&in=state:0" + str(i)
        else:
            url = "https://api.census.gov/data/2010/dec/responserate?get=GEO_ID,FSRR2010&for=tract:*&in=state:" + str(i)
        JSONContent = requests.get(url).json()
        temp = pd.DataFrame(JSONContent)
        temp = temp.iloc[1:,[0,1]]
        temp.columns = columns
        responses = pd.concat([responses,temp])

In [267]:
url_base = 'https://api.census.gov/data/2010/dec/sf1?get=\
GEO_ID,P001001,P002002,P002005,P003002,P003003,P003004,\
P003005,P003006,P003007,P003008,P004002,P004003,\
P016002,P016003,P037001&for=tract:*&in=state:'
url = url_base + '01'
JSONContent = requests.get(url).json()
temp = pd.DataFrame(JSONContent)
columns = temp.iloc[0,:-3]
demos = pd.DataFrame(columns = columns)

for i in range(1,57):
    if i not in [3,7,14,43,52]:
        if i<10:
            url = url_base + '0' + str(i)
        else:
            url = url_base + str(i)
        JSONContent = requests.get(url).json()
        temp = pd.DataFrame(JSONContent)
        temp = temp.iloc[1:,:-3]
        temp.columns=columns
        demos = pd.concat([demos,temp])

In [268]:
demos.index = demos.GEO_ID
demos = demos.drop(columns='GEO_ID')
responses.index = responses.GEO_ID
responses = responses.drop(columns='GEO_ID')

data = responses.merge(demos,how='inner',left_index=True,right_index=True)
data = data[data.FSRR2010.notna()]

data.columns = ['response','pop','urban','rural','white',\
                'black','AIAN','asian','pacific_islander',\
                'other_race','multiracial','non-hispanic','hispanic',\
               'under18','18orolder','avgfamsize']

data = data.astype('float32')

for column in data.columns:
    if column not in ['pop','response','avgfamsize']:
        data[column] = data[column]/data['pop']

        
data = data.fillna(value='0')

data.to_csv('responses.csv')